In [1]:
# Install dependencies
!pip install -q transformers torchaudio librosa accelerate reportlab gradio

Access is denied.


In [1]:
from transformers import pipeline

transcriber = pipeline(
    "automatic-speech-recognition",
    model = "openai/whisper-small",
    device = 0,
)

def transcribe_audio(file_path):
    """
    Transcribe audio file to text using Whisper model.

    Args:
        file_path (str): Path to the audio file.

    Returns:
        str: Transcribed text.
    """
    
    transcription = transcriber(file_path)


    text = transcription['text']

    return text

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample.mp3 to sample.mp3


In [3]:
transcript = transcribe_audio("sample.mp3")  # change if your file has a different name
print("Transcript:\n", transcript)

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


🎧 Transcript:
  Worrying us that should fight it you're no nearer here. It's trance step in which you know shipping a second sword to natural every point in pilot and bends that


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from huggingface_hub import login

import os

hf_token = os.getenv("HF_TOKEN")


login(token=hf_token)

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

summarizer = pipeline("text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
def format_prompt(transcript):
    return f"""
You are an expert meeting assistant. Based on the transcript below, generate:

1. A concise summary of the discussion
2. A list of 3–5 action items
3. Any important decisions made

Transcript:
\"\"\"{transcript}\"\"\"
"""

def summarize_transcript(transcript):
    prompt = format_prompt(transcript)

    response = summarizer(prompt, max_new_tokens=512, do_sample=True, temperature=0.7)
    return response[0]["generated_text"]


In [6]:
summary = summarize_transcript(transcript)
print("Summary:\n", summary)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📝 Summary:
 
You are an expert meeting assistant. Based on the transcript below, generate:

1. A concise summary of the discussion
2. A list of 3–5 action items
3. Any important decisions made

Transcript:
""" Worrying us that should fight it you're no nearer here. It's trance step in which you know shipping a second sword to natural every point in pilot and bends that"""

Summary:
This meeting is a discussion about a problem concerning a second sword that is being shipped to a natural point in pilot. The participants are concerned about the delay in the delivery and the possibility of it not being shipped.

Action Items:

1. Investigate the cause of the delay in the shipment of the second sword.
2. Determine the exact delivery date of the second sword.
3. Ensure that the second sword is properly packaged and labeled for shipping.
4. Follow up with the shipping company regularly to monitor the progress of the shipment.
5. Have a backup plan in place in case the second sword is not deli

In [7]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.9 MB/s eta 0:00:00


In [8]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
import datetime

def generate_summary_pdf(summary_text, output_path="meeting_summary.pdf"):
    """
    Generate a PDF from a summary string.
    """
    c = canvas.Canvas(output_path, pagesize=A4)
    width, height = A4
    c.setFont("Helvetica", 12)

    # Title
    c.drawString(40, height - 40, "📄 Meeting Summary")
    c.setFont("Helvetica", 10)
    c.drawString(40, height - 60, f"Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}")

    # Content
    text = c.beginText(40, height - 100)
    text.setFont("Helvetica", 11)

    for line in summary_text.split("\n"):
        if text.getY() < 40:
            c.drawText(text)
            c.showPage()
            text = c.beginText(40, height - 40)
            text.setFont("Helvetica", 11)
        text.textLine(line.strip())

    c.drawText(text)
    c.save()
    return output_path


In [9]:
pdf_path = generate_summary_pdf(summary)
print("PDF saved at:", pdf_path)

✅ PDF saved at: meeting_summary.pdf


In [10]:
from google.colab import files
files.download(pdf_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>